# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron
- Input Layer
- Hidden Layer
- Output Layer
- Activation
- Backpropagation

* Neuron: A Neuron is a brain / nerve cell that serves as a logic gate in our Brain. An Artificial Neuron is a mathematical function model of the cell. It calculates some equations on the input and decides to return a 1 or 0. In similar fashion a brain neuron receives multiple stimuli and then triggers the sending of a signal when appropriate (and sometimes when not appropriate).

* Input layer: The input layer is the input variables, sometimes called the "visable layer".

* Hidden Layer: The one or more processing layers of varying sizes and configurations that make up the MLP or deep learning model. Hidden layers are required if and only if the data must be separated non-linearly. By adding additional layers we can handle increasingly complex data relationships .

* Output Layer: The collection of output nodes that are responsible for computations and transferring information from the network to the outside world. These evaluate the combinations from the weights and hidden layer outputs and assign a final score. 

* Activation: Activation functions are used to determine how an input signal is pushed forward by a node. It does this by applying the activation function which maps the input value against an established thresholding function that constrains the output within the desired range (usually 0, 1 or -1, 1). By doing so each node can then produce a constrained output.

* Backpropagation:  In backpropagation we take the error (the "cost" that we compute by comparing the calculated output and the known, correct target output) and we then use it to update the model parameters. We calculate the total error at the output nodes and propagate these errors back through the network in each layer. This causes our network to perform better when calculating gradients than before since the weights have now been adjusted to minimize the error in prediction.

## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [13]:
# Start by creating weights and bias
weight1 = 0.3334
weight2 = 0.3334
weight3 = 0.3334
bias = -1


# Add inputs and outputs
test_inputs = [(1, 1, 1), (1, 0, 1), (0, 1, 1), (0, 0, 1), (1, 1, 0), (1, 0, 0), (0,0,0)]
correct_outputs = [True, False, False, False]
outputs = []

# Generate and check output
for test_input, correct_output in zip(test_inputs, correct_outputs):
    linear_combination = weight1 * test_input[0] + weight2 * test_input[1] + weight3 * test_input[2] + bias
    output = int(linear_combination >= 0)
    is_correct_string = 'Yes' if output == correct_output else 'No'
    outputs.append([test_input[0], test_input[1],  test_input[2], linear_combination, output, is_correct_string])

# Print output
output_frame = pd.DataFrame(outputs, columns=['Input 1', '  Input 2', 'Input 3','  Linear Combination', '  Activation Output', '  Is Correct'])
print(output_frame.to_string(index=False))

Input 1    Input 2  Input 3    Linear Combination    Activation Output   Is Correct
      1          1        1                0.0002                    1          Yes
      1          0        1               -0.3332                    0          Yes
      0          1        1               -0.3332                    0          Yes
      0          0        1               -0.6666                    0          Yes


## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


In [18]:
import pandas as pd
import numpy as np
data = "https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv"

df = pd.read_csv(data, header=0)
display(df.shape)
df = df.dropna(axis=0, how='any')
display(df.shape)
display(df.head(5))
display(df.target.value_counts())

(303, 14)

(303, 14)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


1    165
0    138
Name: target, dtype: int64

In [0]:
import sys
import numpy as np

class NeuralNetMLP(object):
    """
    Feed forward nueral network / multi-layer perceptron classifier
    
    Parameters
    ----------
    n_hidden : int (default: 30)
        Number of hidden units
    l2 : float (default 0.)
        Lambda value for l2 normalization
    epochs : int (default: 100)
        Number of training epochs
    eta : float (default = 0.001)
        Learning rate
    shuffle : bool (default: True)
        Shuffles the training data every epoch if True
    minibatch_size : int (default : 1)
        Number of training samples per minibatch
    seed : int (default: None)
        Random seed for initalizing weights and shuffling
    
    Attributes
    ----------
    eval_ : dict
        Dictionary collecting the cost, training accuracy,
        and validation accuracy for each epoch during training
    
    """
    
    def __init__(self, n_hidden=30, l2=0.,
                epochs=100, eta=0.001,
                shuffle=True, minibatch_size=1, seed=None):
        self.random = np.random.RandomState(seed)
        self.n_hidden = n_hidden
        self.l2 = l2
        self.epochs = epochs
        self.eta = eta
        self.shuffle = shuffle
        self.minibatch_size = minibatch_size
        
    def _onehot(self, y, n_classes):
        """
        Encode labels into one hot representation
        
        Parameters
        ----------
        y : array, shape = [n_samples]
            Target values
        
        Returns
        -------
        onehot : array, shape = (n_samples, n_labels)
        """
        
        onehot = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1.
        return onehot.T
    
    def _sigmoid(self, z):
        """
        Compute logistic function (sigmoid)
        """
        return 1. / (1. + np.exp(-np.clip(z, -250, 250)))
    
    def _forward(self, X):
        """
        Compute forward propogation step
        """
        # step 1: net input of hidden layer
        z_h = np.dot(X, self.w_h) + self.b_h
        
        # step 2: activation of hidden layer
        a_h = self._sigmoid(z_h)
        
        # step 3: net input of output layer
        z_out = np.dot(a_h, self.w_out) + self.b_out
        
        # step 4: activation layer output
        a_out = self._sigmoid(z_out)
        
        return z_h, a_h, z_out, a_out
    
    def _compute_cost(self, y_enc, output):
        """
        Compute cost function
        
        Parameters
        ----------
        y_enc : array, shape = (n_samples, n_labels)
            one-hot encoded class labels
        output : array, shape = [n_samples, n_output_units]
            Activation of the output layer (forward propoagation)
        Returns
        -------
        cost : float
            Regularized cost
        """
        e = 0.000000001
        L2_term = (self.l2 *
                  (np.sum(self.w_h ** 2.) + 
                  np.sum(self.w_out ** 2.)))
        term1 = -y_enc * (np.log(output + e))
        term2 = (1. - y_enc) * np.log(1. - output + e)
        cost = np.sum(term1 - term2) + L2_term
        return cost
    
    def predict(self, X):
        """
        Predict class labels
        
        Paramters
        ---------
        X : array, shape = [n_samples, n_features]
            Input layer with original features
        
        Returns
        -------
        y_pred : array, shape = [n_samples]
            Predicted class labels
        """
        z_h, a_h, z_out, a_out = self._forward(X)
        y_pred = np.argmax(z_out, axis=1)
        return y_pred
    
    def fit(self, X_train, y_train, X_valid, y_valid):
        """
        Learn weights from training data
        
        Parameters
        ----------
        X_train : array, shape = [n_samples, n_features]
            Input layer with original features
        y_train : array, shape = [n_samples]
            Target class labels
        X_test : array, shape = [n_samples, n_features]
            Sample features for validation during training
        y_test : array, shape = [n_samples]
            Samples test class labels for validation during training
            
        Returns
        -------
        self
        """
        n_output = np.unique(y_train).shape[0]
        n_features = X_train.shape[1]
        
        #######################
        # Weight Initialization
        #######################
        
        # weights for input -> hidden
        self.b_h = np.zeros(self.n_hidden)
        self.w_h = self.random.normal(loc=0.0,
                                     scale=0.1,
                                     size=(n_features,
                                          self.n_hidden))
        
        # weights for hidden -> output
        self.b_out = np.zeros(n_output)
        self.w_out = self.random.normal(loc=0.0,
                                       scale=0.1,
                                       size=(self.n_hidden,
                                       n_output))
        
        epoch_strlen = len(str(self.epochs)) # for progr. format
        self.eval_ = {'cost' : [],
                     'train_acc' : [],
                     'valid_acc' : []}
        
        y_train_enc = self._onehot(y_train, n_output)
        
        # iterate over training epochs
        for i in range(self.epochs):
            
            # iterate over mini batches
            indices = np.arange(X_train.shape[0])
            
            if self.shuffle:
                self.random.shuffle(indices)
            
            for start_idx in range(0, indices.shape[0] - self.minibatch_size + 1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]
                
                # forward propagation
                z_h, a_h, z_out, a_out = self._forward(X_train[batch_idx])
                
                #################
                # Backpropagation
                #################
                
                # [n_samples, n_classlabels]
                sigma_out = a_out - y_train_enc[batch_idx]
                
                # [n_samples, n_hidden]
                sigmoid_derivative_h = a_h * (1. - a_h)
                
                # [n_samples, n_classlabels] dot [n_classlabels, n_hidden]
                # -> [n_samples, n_hidden]
                sigma_h = (np.dot(sigma_out, self.w_out.T) * sigmoid_derivative_h)
                
                # [n_features, n_samples] dot [n_samples, n_hidden]
                # -> [n_features, n_hidden]
                grad_w_h = np.dot(X_train[batch_idx].T, sigma_h)
                grad_b_h = np.sum(sigma_h, axis=0)
                
                # [n_hidden, n_samples] dot [n_samples, n_classlabels]
                # -> [n_hidden, n_classlabels]
                grad_w_out = np.dot(a_h.T, sigma_out)
                grad_b_out = np.sum(sigma_out, axis=0)
                
                # Regularization and weight updates
                delta_w_h = (grad_w_h + self.l2 * self.w_h)
                delta_b_h = grad_b_h # bias is not regularized
                self.w_h -= self.eta * delta_w_h
                self.b_h -= self.eta * delta_b_h
                
                delta_w_out = grad_w_out + self.l2 * self.w_out
                delta_b_out = grad_b_out # bias is not regularized
                self.w_out -= self.eta * delta_w_out
                self.b_out -= self.eta * delta_b_out
                
            ############
            # Evaluation
            ############

            # evaluation after each epoch during training
            z_h, a_h, z_out, a_out = self._forward(X_train)
            
            cost = self._compute_cost(y_enc=y_train_enc, output=a_out)
            
            y_train_pred = self.predict(X_train)
            y_valid_pred = self.predict(X_valid)
            
            train_acc = ((np.sum(y_train == y_train_pred)).astype(np.float) / X_train.shape[0])
            valid_acc = ((np.sum(y_valid == y_valid_pred)).astype(np.float) / X_valid.shape[0])
            
            sys.stderr.write('\r%0*d/%d | Cost: %.2f | Train/Valid Acc: %.2f%%/%.2f%%'
                            % (epoch_strlen, i+1, self.epochs, cost, train_acc*100, valid_acc*100))
            sys.stderr.flush()
            
            self.eval_['cost'].append(cost)
            self.eval_['train_acc'].append(train_acc)
            self.eval_['valid_acc'].append(valid_acc)
            
        return self

In [0]:
# Initialize a new MLP
mlp = NeuralNetMLP(n_hidden= 26,
                   l2= 0.01,
                   epochs= 200,
                   eta= 0.0005,
                   minibatch_size= 100,
                   shuffle = True,
                   seed = 42)

In [25]:
from sklearn.preprocessing import MinMaxScaler

X = df.drop(columns = "target")
X = MinMaxScaler().fit_transform(X)
y = df.target.copy()

display(X.shape)
display(y.shape)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:334: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


(303, 13)

(303,)

## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [0]:
##### Your Code Here #####